0. Выкинул ~3000 изображений, со смещенной разметкой. Можно было им сдвинуть разметку в правильное место и тоже использовать, но решил, что их всего 1%.
1. Аугментация - RandomCrop - случайным образом выбирал размер квадрата, который больше landmark и меньше, чем картинка. И случайным образом смещал его относительно разметки, но чтобы она осталась в нём. (такая была только на обучении, на валидации и тесте - из бейзлайна)
2. AdamW
3. обучался на MSE из бейзлайна, а на валидации метрику пересчитывал с учётом трансформации.
4. Эпох = пока падали метрики ~70
5. ResNet50, не морозил.

Не делал или не зашло:
С цветами, яркостью и блюром почему-то не стал играть. Может и зря.  
18-34-50 - давали улучшения. ResNet101 - как-то не пошёл. Пробовал другие предобученые модели - не было той, чтобы понравилось. Искал что-нибудь, чтобы быстрее училась.  
Пробовал Flip начальной картинки - для этого сделал корректный пересчёт точек, но сначала сделал все медленно и как-то не давало улучшения, потом переделал чтобы быстро работало. Явных улучшений не было, но были странные расшатывания модели. Когда на очередной итерации скор мог скакнуть сильно.  
Хотел как-нибудь использовать keypoint - но предобученая модель давала странный результат (находила на 1 лице несколько комплектов точек) и забросил.  
Пробовал в конце 2 FC слоя с дропаутом - не понравилось.  
Пробовал учить только 1 последний слой - тоже не зашло.  
Была мысль, что может зря выкидываю изображения со смещенной разметкой при обучении. Вдруг в тесте тоже такие будут. Но потом решил, что там такой дикий скор получается на них, что нет смысла. Да и задача у нас все-таки правильную разметку строить, а не угадывать когда она сдвинута.

In [ ]:
import os
import numpy as np
import cv2
import tqdm.notebook as tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.nn import functional as fnn
from torch.utils import data
from torchvision import transforms

from utils import NUM_PTS, CROP_SIZE
from utils import train,validate,predict
from utils import ScaleMinSideToSize,CropCenter,TransformByKeys
from utils import ThousandLandmarksDataset
from utils import restore_landmarks_batch, create_submission,restore_landmarks_batch_ex,draw_landmarks

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
use_gpu = True
data_size = None

In [ ]:
fn_ignore = "ignore_images.lst"
data_dir = "C:/_Data/full/"
batch_size = 192
learning_rate = 1e-3
epochs1 = 30
epochs2 = 40
prj_name = "test10"
# data_size = 40000
# data_size = 300

In [ ]:
# метрика, которая учитывает масштабирование изображений - для отображения при валидации
class MseW(torch.nn.Module):
    def __init__(self):
        super(MseW,self).__init__()

    def setWeight(self, weight):
        self.w = weight
        
    def forward(self, outputs, labels):
        mse = torch.mul(outputs - labels,outputs - labels).mean(axis=1)        
        mse=torch.mul(mse,self.w).mean(axis=0)
        mse=mse.mean(axis=0)
        mse=2*mse
        return mse        

In [ ]:
class CropRandom(object):
    def __init__(self, size=CROP_SIZE, elem_name='image'):
        self.size = torch.tensor(size, dtype=torch.float)
        self.elem_name = elem_name

    def __call__(self, sample):
        if 'landmarks' in sample:
            img = sample[self.elem_name] #.copy()
            landmarks = sample['landmarks'].reshape(-1, 2)
            bound=landmarks[:,0].min(),landmarks[:,1].min(),landmarks[:,0].max(),landmarks[:,1].max()                        
            h,w,_ = img.shape
            min_sq = max(bound[3]-bound[1],bound[2]-bound[0])
            max_sq = min(w,h)            
            if min_sq+1<max_sq-1:
                sq = np.random.randint(min_sq+1,max_sq-1)
            else:
                sq = max_sq-1                
            
            min_dx = max(bound[2]-sq,0)
            max_dx = min(w-sq,bound[0])
            if min_dx<max_dx:
                dx = np.random.randint(min_dx,max_dx)
            else:
                dx = int(min_dx)
            
            min_dy = max(bound[3]-sq,0)
            max_dy = min(h-sq,bound[1])
            if min_dy<max_dy:
                dy = np.random.randint(min_dy,max_dy)
            else:                
                dy = int(min_dy)
                
            landmarks -= torch.tensor((dx, dy), dtype=landmarks.dtype)[None, :]                        
            sample['landmarks'] = landmarks.reshape(-1)
            sample[self.elem_name] = img[dy:dy+sq, dx:dx+sq]
            sample['dx'] = torch.tensor(dx,dtype=torch.short)
            sample['dy'] = torch.tensor(dy,dtype=torch.short)
        else:
            raise RuntimeError(f"stop")
            sample['dx'] = torch.tensor(0)
            sample['dy'] = torch.tensor(0)
        return sample    
       
# class RandomFlipV(object):
#     def __init__(self, size=128, elem_name='image'):
#         self.size = size
#         self.elem_name = elem_name

#     def __call__(self, sample):
#         if np.random.randint(0,10)>4:
#             sample['flip'] = True                    
#             img = sample[self.elem_name]            
#             sample[self.elem_name] = img[:,::-1,:]
#             if 'landmarks' in sample:
#                 landmarks = sample['flip_landmarks']                
#                 landmarks[:,0] = img.shape[1]-landmarks[:,0] 
#                 sample['landmarks'] = landmarks #torch.as_tensor(flip_lm_v(landmarks,img.shape))
#         else:
#             sample['flip'] = False
#         return sample 

In [ ]:
ignore_image = set()
with open(fn_ignore, "rt") as fp:
    for line in fp:
        ignore_image.add(line.strip())
print(len(ignore_image))

In [ ]:
train_loss_fn = fnn.mse_loss
valid_loss_fn = MseW()

In [ ]:
device = torch.device("cuda: 0") if use_gpu else torch.device("cpu")

In [ ]:
# pipeline при обучении
train_transforms = transforms.Compose([
#     RandomFlipV(),
    CropRandom(),    
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
])
# pipeline при валидации и предсказании
val_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
])

In [ ]:
%%time
print("Reading data...")
train_dataset = ThousandLandmarksDataset(os.path.join(data_dir, 'train'), train_transforms, split="train",size = data_size,ignore_image=ignore_image)
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, pin_memory=True,drop_last=True,shuffle=True)
print(len(train_dataset))

In [ ]:
val_dataset = ThousandLandmarksDataset(os.path.join(data_dir, 'train'), val_transforms, split="val",size = data_size,ignore_image=ignore_image)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, num_workers=0, pin_memory=True,drop_last=False,shuffle=False)
print(len(val_dataset))

In [ ]:
print("Creating model...")
device = torch.device("cuda: 0") if use_gpu else torch.device("cpu")
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, amsgrad=True)

In [ ]:
# 2. train & validate
print("Ready for training...")
best_val_loss = np.inf
for epoch in range(0,epochs1):    
    train_loss = train(model, train_dataloader, train_loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, valid_loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.4f}\tval loss: {:5.4f}".format(epoch, train_loss, val_loss))    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"{prj_name}_best.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)
    with open(f"{prj_name}_"+str(epoch)+".pth", "wb") as fp:
            torch.save(model.state_dict(), fp)


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, amsgrad=True)

In [ ]:
# 2. train & validate
print("Ready for training...")
best_val_loss = np.inf
for epoch in range(epochs1,epochs1+epochs2):    
    train_loss = train(model, train_dataloader, train_loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, valid_loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.4f}\tval loss: {:5.4f}".format(epoch, train_loss, val_loss))    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"{prj_name}_best.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)
    with open(f"{prj_name}_"+str(epoch)+".pth", "wb") as fp:
            torch.save(model.state_dict(), fp)


In [ ]:
# 3. predict last
test_dataset = ThousandLandmarksDataset(os.path.join(data_dir, 'test'), val_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, pin_memory=True,
                                  shuffle=False, drop_last=False)

test_predictions = predict(model, test_dataloader, device)
with open(f"{prj_name}_last_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

create_submission(data_dir, test_predictions, f"{prj_name}_last_submit.csv")


In [ ]:
# predict best
with open(f"{prj_name}_best.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"{prj_name}_best_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

create_submission(data_dir, test_predictions, f"{prj_name}_best_submit.csv")
